# Falling Ball

## Import packages

In [1]:
import cv2
import numpy as np
import math

## Load background and ball images

In [2]:
back_img = cv2.imread('./imgs/high-contrast.tif')
ball_img = cv2.imread('./imgs/basketball.png')

## Initial Parameters

In [3]:
fps = 30
#Time interval between two frames
t = 1000 // fps
# Initial velocity along the x and y axes
initial_velocity = np.array([5, 10], dtype=float)  # Initial velocity (x, y)
gravity = 1
bounce_value = 0.8
ball_height, ball_width = 30, 30

## Normalize Images

In [4]:
back_img = cv2.normalize(back_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
ball_img = cv2.normalize(ball_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

## Resize ball image 
**Resize ball image base on ball_height and ball_width parameters**

In [5]:
ball_img = cv2.resize(ball_img, None, fx=ball_width / ball_img.shape[1], fy=ball_height / ball_img.shape[0],
                      interpolation=cv2.INTER_AREA)

# Make masks
**In order to create a mask and create a transparent ball correctly, the background of the ball image must be white.** \
**In this example, the alpha channel is not used.**

In [6]:
back_mask = (ball_img[..., 0] > 0.8) & (ball_img[..., 1] > 0.8) & (ball_img[..., 1] > 0.8)
back_mask = np.array(back_mask, np.float32)
back_mask = cv2.cvtColor(back_mask, cv2.COLOR_GRAY2BGR)
img_mask = 1 - back_mask

## Initialization of computational parameters

In [7]:
ball_radius = ball_height // 2
width, height = back_img.shape[1], back_img.shape[0]
initial_position = (width // 2, 0)
rim_height = height - ball_radius * 2  # Height of the rim

# Start of simulate ball motion

In [8]:
position = np.array(initial_position, dtype=float)
velocity = initial_velocity.copy()

lastvelocity = 0
while 0 <= position[0] < width and 0 <= position[1] < height and math.trunc(lastvelocity * 10000000) != math.trunc(
        velocity[1] * 10000000):
    # Update position and velocity
    if position[1] != 0:
        back_img[int(position[1]):int(position[1]) + ball_img.shape[0],
        int(position[0]):int(position[0]) + ball_img.shape[1]] = back

    lastvelocity = velocity[1]
    velocity[1] += gravity
    position += velocity

    # Bounce off the rim
    if position[1] >= rim_height:
        position[1] = rim_height
        velocity[1] *= -bounce_value  # Reverse vertical velocity with some damping

    # Bounce off left and right edges
    if position[0] <= 0 or position[0] >= width - ball_radius * 2:
        velocity[0] *= -1  # Reverse horizontal velocity

    # Copy background of ball
    back = np.copy(back_img[int(position[1]):int(position[1]) + ball_img.shape[0],
                   int(position[0]):int(position[0]) + ball_img.shape[1]])
    # Rotate ball
    ball_img = image = cv2.rotate(ball_img, cv2.ROTATE_90_CLOCKWISE)
    # Draw the ball with transparency
    back_img[int(position[1]):int(position[1]) + ball_img.shape[0],
    int(position[0]):int(position[0]) + ball_img.shape[1]] = ball_img * img_mask + back * back_mask

    # Display the frame
    cv2.imshow("Falling Ball", back_img)
    key = cv2.waitKey(t)
    # Exit by pressing the Esc or q key
    if key == 27 or key == ord("q"):
        break
cv2.destroyAllWindows()        